In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np
import json

# Read the JavaScript data from the file
with open('main.js', 'r') as file:
    js_code = file.read()


# Extract the JavaScript array parts
start_index1 = js_code.find('[')
end_index1 = js_code.find(']', start_index1)
js_array_str1 = js_code[start_index1:end_index1 + 1]

start_index2 = js_code.find('[', end_index1)
end_index2 = js_code.find(']', start_index2)
js_array_str2 = js_code[start_index2:end_index2 + 1]

# Convert the JavaScript array strings to Python lists
numbers1 = json.loads(js_array_str1)
numbers2 = json.loads(js_array_str2)

print("Red Team:", numbers1)  # Output: [1, 2, 3, 4, 5]
print("Blue Team:", numbers2)  # Output: [6, 7, 8, 9, 10]


Red Team: ['Phoenix', 'Clove', 'Reyna', 'Skye', 'Cypher']
Blue Team: ['Clove', 'Sova', 'Reyna', 'Neon', 'Cypher']


In [18]:
import requests

# Make a GET request to the API endpoint
response = requests.get('https://api.henrikdev.xyz/valorant/v3/matches/na/angi/chi')
data = ""

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Convert the response to JSON format
    data = response.json()
    # Now 'data' contains the JSON response from the API
    # print(data)
else:
    print('Failed to retrieve data from the API:', response.status_code)

# Step 1: Data Preparation
import pandas as pd

parsed_data = data

# Initialize a dictionary to store the best agent for each map
best_agents_for_maps = {}

# Loop through each match
for match in parsed_data['data']:
    # Extract map name
    map_name = match['metadata']['map']
    
    # Extract players and their characters
    players = match['players']['all_players']
    
    # Initialize a dictionary to store performance metrics for each character on the map
    character_performance = {}
    
    # Analyze performance of each character on the map
    for player in players:
        character = player['character']
        if character not in character_performance:
            character_performance[character] = {'wins': 0, 'kills': 0, 'deaths': 0, 'assists': 0}
        
        # Update performance metrics
        if match['teams']['red']['has_won'] and player['team'] == 'Red':
            character_performance[character]['wins'] += 1
        character_performance[character]['kills'] += player['stats']['kills']
        character_performance[character]['deaths'] += player['stats']['deaths']
        character_performance[character]['assists'] += player['stats']['assists']
    
    # Determine the best character to play on the map based on performance metrics
    best_character = max(character_performance, key=lambda x: character_performance[x]['wins'])
    
    # Store the best agent for the map
    best_agents_for_maps[map_name] = best_character

# Print the best agent for each map
print("Best agent to play for each map:")
for map_name, best_agent in best_agents_for_maps.items():
    print(map_name + ":", best_agent)

Best agent to play for each map:
Bind: Clove
Ascent: Iso
Lotus: Killjoy
Split: Sova
